In [1]:
import pickle
import pandas as pd
import os
# os.chdir('../src')

In [3]:
df = pickle.load(open('../data/intermediate/patient_demographics.pkl','rb'))
visits_data = pickle.load(open('../data/intermediate/visits_data.pkl','rb'))
disease_list = visits_data.DiseaseCat.unique().tolist() # ['major depressive disorder']
comorbids = ['anxiety disorders', 'dissociative disorders']

# Filtering

In [272]:
def add_filt(filt_values, demog=df, visits=visits_data, age_band=5):
    """For adding demographics filters to the dataframe.
    """
    demog_filters = []
    visit_filters = []
    
    for filt, val in filt_values.items():
        if val != None:
            print(filt, val)
            if filt=='Medication' or filt=='DiseaseCat':
                if not val == 'all':
                    if not isinstance(val, list): val = [val]
                    f = visits[filt].isin(val)
                    visit_filters.append(f)
            
            elif filt=='Age':
                f = abs(demog['Age']-filt_values['Age'])<=age_band
                demog_filters.append(f)

            else: 
                f = demog[filt]== val
                demog_filters.append(f)

    demogs = demog[pd.concat(demog_filters, axis=1).all(axis=1)]
    visit_filters.append(visits.PatientID.isin(demogs.PatientID))
    filteredDF = visits[pd.concat(visit_filters,axis=1).all(axis=1)]
    
    return filteredDF

In [273]:
len(x.PatientID.unique())

151

In [281]:
visits_data

,PatientID,VisitID,Days,VisitType,CGI,Medication,Dose,Regimen,Diagnosis,DSMNo,DiseaseCat
0,92,1667,105,Outpatient,2,fluoxetine,20.0,1.0,Diagnosis Deferred on Axis II,799.9,others
1,92,1668,105,Outpatient,1,fluoxetine,20.0,1.0,Bulimia Nervosa,307.51,feeding and eating disorders
2,92,1669,104,Outpatient,1,fluoxetine,20.0,1.0,Opioid Dependence,304.00,substance-related and addictive disorders
3,92,1673,104,Outpatient,1,fluoxetine,20.0,1.0,Opioid Dependence,304.00,substance-related and addictive disorders
4,92,1674,103,Outpatient,1,fluoxetine,20.0,1.0,Diagnosis Deferred on Axis II,799.9,others
5,92,1675,103,Outpatient,1,fluoxetine,20.0,1.0,Diagnosis Deferred on Axis II,799.9,others
6,92,1677,103,Outpatient,2,fluoxetine,20.0,1.0,Depressive Disorder NOS,311,other mood disorder
7,92,1679,102,Outpatient,1,fluoxetine,20.0,1.0,Diagnosis Deferred on Axis II,799.9,others
8,92,1681,102,Outpatient,1,fluoxetine,20.0,1.0,Diagnosis Deferred on Axis II,799.9,others
9,92,1691,99,Outpatient,1,fluoxetine,20.0,1.0,Bulimia Nervosa,307.51,feeding and eating disorders


In [282]:
## Selected Values; if not Chosen, value = None. Else value = value of checklist. 
filt_values = {'Sex':'M',
                'Age': None,
                'Race': 'white',
                'Medication':'fluoxetine', # default: all ticked.
               'DiseaseCat' : ['trauma and stressor related disorders'] # default: all ticked. 
               }

filtered_df = add_filt(filt_values, df, visits_data)
print(filtered_df.shape)
filtered_df.head()

Sex M
Race white
Medication fluoxetine
DiseaseCat ['trauma and stressor related disorders']
(255, 11)


,PatientID,VisitID,Days,VisitType,CGI,Medication,Dose,Regimen,Diagnosis,DSMNo,DiseaseCat
78176,108248,2638488,395,Outpatient,2,fluoxetine,40.0,1.0,Adjustment Disorder Unspecified,309.9,trauma and stressor related disorders
78177,108248,2638489,372,Outpatient,2,fluoxetine,40.0,1.0,Adjustment Disorder Unspecified,309.9,trauma and stressor related disorders
78178,108248,2638490,358,Outpatient,2,fluoxetine,40.0,1.0,Adjustment Disorder Unspecified,309.9,trauma and stressor related disorders
78180,108248,2638492,324,Outpatient,2,fluoxetine,40.0,1.0,Adjustment Unspecified,309.9,trauma and stressor related disorders
78181,108248,2638493,308,Outpatient,2,fluoxetine,40.0,1.0,Adjustment Unspecified,309.9,trauma and stressor related disorders


In [225]:
len(filtered_df.PatientID.unique())

151